In [ ]:
# Импорты и настройки
import asyncio
import asyncpg
import pandas as pd
import json
import os
from datetime import datetime
from pathlib import Path
from collections import Counter
from typing import List, Dict, Any

In [ ]:
# Конфигурация
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "news_analyzer",
    "user": "",  # Измените на ваш PostgreSQL пользователь
    "password": "",  # Измените на ваш PostgreSQL пароль
}

# Настройки экспорта
OUTPUT_DIR = "./kaggle_dataset"
MAX_SAMPLES = 10000

# Настройки фильтрации
MIN_FREQ_TRAIN = 14  # Минимальная частота хештега для train
MIN_FREQ_TEST = 6  # Минимальная частота хештега для test

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"📁 Папка для экспорта: {OUTPUT_DIR}")
print(
    f"🔧 Настройки фильтрации: train_freq={MIN_FREQ_TRAIN}, test_freq={MIN_FREQ_TEST}"
)

In [ ]:
# Функции для загрузки данных
async def load_training_data():
    """Загружает данные для обучения из PostgreSQL"""
    try:
        print("🔌 Подключение к PostgreSQL...")
        conn = await asyncpg.connect(**DB_CONFIG)

        query = """
        SELECT 
            m.text,
            a.hashtags,
            a.sentiment,
            m.channel_title
        FROM messages m
        JOIN analyses a ON m.message_id = a.message_id
        WHERE 
            LENGTH(m.text) BETWEEN 50 AND 2000
            AND a.hashtags IS NOT NULL
            AND a.hashtags != '[]'
            AND a.hashtags != 'null'
            AND jsonb_array_length(a.hashtags::jsonb) BETWEEN 2 AND 10
        ORDER BY m.date DESC
        LIMIT $1
        """

        print(f"📊 Загружаем данные (макс. {MAX_SAMPLES})...")
        rows = await conn.fetch(query, MAX_SAMPLES)
        await conn.close()

        print(f"✅ Загружено {len(rows)} записей")
        return rows

    except Exception as e:
        print(f"❌ Ошибка подключения: {e}")
        print("💡 Убедитесь, что PostgreSQL запущен и доступен")
        return []

In [ ]:
# Функции для подготовки данных
def prepare_training_examples(raw_data):
    """Подготавливает примеры в формате ChatML для Saiga LLaMA3"""
    training_examples = []
    system_prompt = "Ты эксперт по генерации хештегов для новостей. Сгенерируй 3-5 релевантных хештегов на русском языке."

    for row in raw_data:
        try:
            hashtags = json.loads(row["hashtags"])
            hashtags_str = ", ".join(hashtags)

            # Формат ChatML для Saiga LLaMA3
            full_prompt = f"<s>system\\n{system_prompt}</s>\\n<s>user\\n{row['text']}</s>\\n<s>assistant\\n{hashtags_str}</s>"

            training_examples.append(
                {
                    "text": full_prompt,
                    "input": row["text"],
                    "output": hashtags_str,
                    "channel": row["channel_title"],
                    "sentiment": row["sentiment"],
                }
            )

        except Exception as e:
            print(f"⚠️ Ошибка обработки записи: {e}")
            continue

    print(f"✅ Подготовлено {len(training_examples)} примеров")
    return training_examples

In [ ]:
# Функции для фильтрации данных
def filter_hashtags_by_frequency(data: List[Dict], min_freq: int = 14) -> List[Dict]:
    """Фильтрует хештеги по частоте встречаемости"""

    def flatten(items):
        """Разворачивает вложенные списки"""
        for x in items:
            if isinstance(x, list):
                yield from flatten(x)
            else:
                yield x

    # 1. Подсчитываем частоты хештегов
    counter = Counter()
    for item in data:
        output = item.get("output", "")
        if output:
            hashtags = [tag.strip() for tag in output.split(",") if tag.strip()]
            counter.update(hashtags)

    # 2. Определяем частые хештеги
    frequent_tags = {tag for tag, freq in counter.items() if freq >= min_freq}
    print(f"📊 Найдено {len(frequent_tags)} хештегов с частотой >= {min_freq}")

    # 3. Фильтруем данные
    filtered_data = []
    for item in data:
        output = item.get("output", "")
        if output:
            hashtags = [tag.strip() for tag in output.split(",") if tag.strip()]
            filtered_hashtags = [tag for tag in hashtags if tag in frequent_tags]

            if filtered_hashtags:  # Если есть хештеги после фильтрации
                new_output = ", ".join(filtered_hashtags)

                # Обновляем output
                filtered_item = item.copy()
                filtered_item["output"] = new_output

                # Синхронизируем assistant-часть в text
                if "text" in filtered_item:
                    parts = filtered_item["text"].rsplit("<s>assistant\\n", 1)
                    if len(parts) == 2:
                        filtered_item["text"] = (
                            parts[0] + "<s>assistant\\n" + new_output + "</s>"
                        )

                filtered_data.append(filtered_item)

    print(f"✅ После фильтрации: {len(filtered_data)} записей")
    return filtered_data

In [ ]:
# Функция для сохранения данных
def save_final_dataset(train_data, test_data, output_dir=OUTPUT_DIR):
    """Сохраняет финальный датасет"""

    # Создаем папку
    os.makedirs(output_dir, exist_ok=True)

    # Сохраняем данные в JSONL формате
    train_file = Path(output_dir) / "train_data_final.jsonl"
    test_file = Path(output_dir) / "test_data_final.jsonl"

    with open(train_file, "w", encoding="utf-8") as f:
        for item in train_data:
            f.write(json.dumps(item, ensure_ascii=False) + "\\n")

    with open(test_file, "w", encoding="utf-8") as f:
        for item in test_data:
            f.write(json.dumps(item, ensure_ascii=False) + "\\n")

    # Конфигурация для Kaggle
    config = {
        "dataset_info": {
            "total_samples": len(train_data) + len(test_data),
            "train_samples": len(train_data),
            "test_samples": len(test_data),
            "created_at": datetime.now().isoformat(),
            "filtered": True,
            "min_freq_train": MIN_FREQ_TRAIN,
            "min_freq_test": MIN_FREQ_TEST,
        },
        "model_config": {
            "base_model": "IlyaGusev/saiga_llama3_8b",
            "task": "hashtag_generation",
            "format": "ChatML",
        },
        "training_config": {
            "epochs": 3,
            "batch_size": 4,
            "learning_rate": 2e-4,
            "lora_r": 16,
            "lora_alpha": 32,
        },
    }

    config_file = Path(output_dir) / "config.json"
    with open(config_file, "w", encoding="utf-8") as f:
        json.dump(config, f, ensure_ascii=False, indent=2)

    # README
    readme_content = f"""# Финальный датасет для дообучения Saiga LLaMA3

- Всего записей: {len(train_data) + len(test_data)}
- Обучающих: {len(train_data)}
- Тестовых: {len(test_data)}
- Фильтрация: Да (min_freq_train={MIN_FREQ_TRAIN}, min_freq_test={MIN_FREQ_TEST})

- `train_data_final.jsonl` - обучающие данные (отфильтрованные)
- `test_data_final.jsonl` - тестовые данные (отфильтрованные)
- `config.json` - конфигурация обучения

Данные готовы для загрузки в Kaggle и обучения модели.
"""

    readme_file = Path(output_dir) / "README.md"
    with open(readme_file, "w", encoding="utf-8") as f:
        f.write(readme_content)

    print(f"✅ Данные сохранены в {output_dir}/")
    print(f"📄 Файлы:")
    print(f"   - train_data_final.jsonl ({len(train_data)} записей)")
    print(f"   - test_data_final.jsonl ({len(test_data)} записей)")
    print(f"   - config.json")
    print(f"   - README.md")

    return {
        "train_samples": len(train_data),
        "test_samples": len(test_data),
        "total_samples": len(train_data) + len(test_data),
    }

In [ ]:
# Основной pipeline - Этап 1: Загрузка данных
print("🚀 Начинаем единый pipeline подготовки данных")
print("=" * 50)

# Загружаем данные из PostgreSQL
raw_data = await load_training_data()

if not raw_data:
    print("Не удалось загрузить данные из PostgreSQL")
    print("Проверьте подключение к базе данных и настройки")
else:
    print(f"Этап 1 завершен: загружено {len(raw_data)} записей")

In [ ]:
# Этап 2: Подготовка в формат ChatML
if raw_data:
    print("\\nЭтап 2: Подготовка данных в формат ChatML")
    training_examples = prepare_training_examples(raw_data)

    if training_examples:
        print(f"Этап 2 завершен: подготовлено {len(training_examples)} примеров")

        # Показываем пример
        print("\\nПример подготовленных данных:")
        print(f"Input: {training_examples[0]['input'][:100]}...")
        print(f"Output: {training_examples[0]['output']}")
        print(f"Text format: {training_examples[0]['text'][:150]}...")
    else:
        print("Не удалось подготовить примеры")
        training_examples = []

In [ ]:
# Этап 3: Разделение на train/test и фильтрация
if training_examples:
    print("\\nЭтап 3: Разделение на train/test и фильтрация")

    # Разделяем на train/test (80/20)
    split_index = int(len(training_examples) * 0.8)
    train_data_raw = training_examples[:split_index]
    test_data_raw = training_examples[split_index:]

    print(f"Разделение: {len(train_data_raw)} train, {len(test_data_raw)} test")

    # Фильтруем хештеги по частоте
    print("\\nФильтрация хештегов по частоте...")
    print(f"Train данные (min_freq={MIN_FREQ_TRAIN}):")
    train_data_filtered = filter_hashtags_by_frequency(train_data_raw, MIN_FREQ_TRAIN)

    print(f"\\nTest данные (min_freq={MIN_FREQ_TEST}):")
    test_data_filtered = filter_hashtags_by_frequency(test_data_raw, MIN_FREQ_TEST)

    print(f"\\nЭтап 3 завершен:")
    print(
        f"   - Train: {len(train_data_filtered)} записей (было {len(train_data_raw)})"
    )
    print(f"   - Test: {len(test_data_filtered)} записей (было {len(test_data_raw)})")

    # Показываем статистику фильтрации
    train_reduction = (
        (len(train_data_raw) - len(train_data_filtered)) / len(train_data_raw) * 100
    )
    test_reduction = (
        (len(test_data_raw) - len(test_data_filtered)) / len(test_data_raw) * 100
    )
    print(
        f"   - Фильтрация удалила: {train_reduction:.1f}% train, {test_reduction:.1f}% test"
    )

In [ ]:
# Этап 4: Сохранение финального датасета
if "train_data_filtered" in locals() and "test_data_filtered" in locals():
    print("\\nЭтап 4: Сохранение финального датасета")

    result = save_final_dataset(train_data_filtered, test_data_filtered)

    print(f"\\nPipeline завершен успешно!")
    print("=" * 50)
    print("Итоговая статистика:")
    print(f"   - Всего записей: {result['total_samples']}")
    print(f"   - Обучающих: {result['train_samples']}")
    print(f"   - Тестовых: {result['test_samples']}")
    print(f"   - Фильтрация применена: Да")
    print(f"   - Формат: ChatML для Saiga LLaMA3")
    print("\\n Следующие шаги:")
    print("   1. Загрузите файлы из kaggle_dataset/ в Kaggle")
    print("   2. Запустите обучение модели")
    print("   3. Интегрируйте обученную модель в Ollama")

else:
    print("Pipeline не может быть завершен из-за ошибок на предыдущих этапах")